In [ ]:
cd /home/ubuntu/FedEM/

In [2]:
# Import General Libraries
import os
import argparse
import torch
import copy
import pickle
import random
import numpy as np
import pandas as pd

# Import FedEM based Libraries
from utils.utils import *
from utils.constants import *
from utils.args import *
from run_experiment import *
from models import *

# Import Transfer Attack
from transfer_attacks.Personalized_NN import *
from transfer_attacks.Params import *
from transfer_attacks.Transferer import *
from transfer_attacks.Args import *
from transfer_attacks.TA_utils import *
from transfer_attacks.Boundary_Transferer import *

np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})


In [ ]:
# Generating Empty Aggregator to be loaded 

setting = 'FedEM'

if setting == 'FedEM':
    nL = 3
else:
    nL = 1
    
# Manually set argument parameters
args_ = Args()
args_.experiment = "cifar10"
args_.method = setting
args_.decentralized = False
args_.sampling_rate = 1.0
args_.input_dimension = None
args_.output_dimension = None
args_.n_learners= nL
args_.n_rounds = 10
args_.bz = 128
args_.local_steps = 1
args_.lr_lambda = 0
args_.lr =0.03
args_.lr_scheduler = 'multi_step'
args_.log_freq = 10
args_.device = 'cuda'
args_.optimizer = 'sgd'
args_.mu = 0
args_.communication_probability = 0.1
args_.q = 1
args_.locally_tune_clients = False
args_.seed = 1234
args_.verbose = 1
args_.save_path = 'weights/cifar/dummy/'
args_.validation = False

# Generate the dummy values here
try:
    aggregator
except:
    aggregator, clients = dummy_aggregator(args_, num_user=40)

try:
    dataloader
except:
    # Compiling Dataset from Clients
    # Combine Validation Data across all clients as test
    data_x = []
    data_y = []

    for i in range(len(clients)):
        daniloader = clients[i].test_iterator
        for (x,y,idx) in daniloader.dataset:
            data_x.append(x)
            data_y.append(y)

    data_x = torch.stack(data_x)
    try:
        data_y = torch.stack(data_y)        
    except:
        data_y = torch.FloatTensor(data_y) 

    dataloader = Custom_Dataloader(data_x, data_y)
    global_dataloader = copy.deepcopy(dataloader)
    
# del aggregator, clients

In [4]:
def get_models_test(directory_name, num_models):
    # Generate the dummy values here
    aggregator, clients = dummy_aggregator(args_, num_user=num_models)
    
    # Combined version
    args_.save_path = directory_name
    weights = np.load(directory_name+'train_client_weights.npy')
    aggregator.load_state(args_.save_path)
    
    # This is where the models are stored -- one for each mixture --> learner.model for nn
    hypotheses = aggregator.global_learners_ensemble.learners

    # obtain the state dict for each of the weights 
    weights_h = []
    
    for h in hypotheses:
        weights_h += [h.model.state_dict()]

    # Set model weights
    model_weights = []

    for i in range(num_models):
        model_weights += [weights[i]]

    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for (w0,w1,w2) in model_weights:
        # first make the model with empty weights
        new_model = copy.deepcopy(hypotheses[0].model)
        new_model.eval()
        new_weight_dict = copy.deepcopy(weights_h[0])
        for key in weights_h[0]:
            new_weight_dict[key] = w0*weights_h[0][key] + w1*weights_h[1][key] + w2*weights_h[2][key]
        new_model.load_state_dict(new_weight_dict)
        models_test += [new_model]
        
    # For h_s
    model_weights = [(1,0,0), (0,1,0), (0,0,1)]
    h_s = []
    for (w0,w1,w2) in model_weights:
        # first make the model with empty weights
        new_model = copy.deepcopy(hypotheses[0].model)
        new_model.eval()
        new_weight_dict = copy.deepcopy(weights_h[0])
        for key in weights_h[0]:
            new_weight_dict[key] = w0*weights_h[0][key] + w1*weights_h[1][key] + w2*weights_h[2][key]
        new_model.load_state_dict(new_weight_dict)
        h_s += [new_model]
    
    del aggregator
    
    return weights, models_test, h_s, clients

def generate_empty_logs_adv(num_models):
    
    # Here we will make a dictionary that will hold results
    logs_adv = []

    for i in range(num_models):
        adv_dict = {}
        adv_dict['orig_acc_transfers'] = None
        adv_dict['orig_similarities'] = None
        adv_dict['adv_acc_transfers'] = None
        adv_dict['adv_similarities_target'] = None
        adv_dict['adv_similarities_untarget'] = None
        adv_dict['adv_target'] = None
        adv_dict['adv_miss'] = None
        adv_dict['metric_alignment'] = None
        adv_dict['ib_distance_legit'] = None
        adv_dict['ib_distance_adv'] = None

        logs_adv += [adv_dict]
    
    return logs_adv
        
# Perform transfer attack from one client to another and record stats

def adv_atk(models_test, num_models, clients, custom_batch_size = 500):
    
    # Run Measurements for both targetted and untargeted analysis
    new_num_models = len(models_test)
    victim_idxs = range(new_num_models)
    custom_batch_size = 500
    eps = 4.5
    
    # Record number of batch sizes
    batch_size_recs = np.ones([num_models,num_models])
    victim_idxs = range(num_models)
    logs_adv = generate_empty_logs_adv(num_models)
    
    print("Running Adv Attack")
    for adv_idx in victim_idxs:
        print("\t Adv idx:", adv_idx)

        dataloader = load_client_data(clients = clients, c_id = adv_idx, mode = 'test') # or test/train
        batch_size = min(custom_batch_size, dataloader.y_data.shape[0])
        batch_size_recs[adv_idx,:] *= batch_size

        t1 = Transferer(models_list=models_test, dataloader=dataloader)
        t1.generate_victims(victim_idxs)

        # Perform Attacks Targeted
        t1.atk_params = PGD_Params()
        t1.atk_params.set_params(batch_size=batch_size, iteration = 10,
                       target = 3, x_val_min = torch.min(data_x), x_val_max = torch.max(data_x),
                       step_size = 0.01, step_norm = "inf", eps = eps, eps_norm = 2)



        t1.generate_advNN(adv_idx)
        t1.generate_xadv(atk_type = "pgd")
        t1.send_to_victims(victim_idxs)

        # Log Performance
        logs_adv[adv_idx]['orig_acc_transfers'] = copy.deepcopy(t1.orig_acc_transfers)
        logs_adv[adv_idx]['orig_similarities'] = copy.deepcopy(t1.orig_similarities)
        logs_adv[adv_idx]['adv_acc_transfers'] = copy.deepcopy(t1.adv_acc_transfers)
        logs_adv[adv_idx]['adv_similarities_target'] = copy.deepcopy(t1.adv_similarities)        
        logs_adv[adv_idx]['adv_target'] = copy.deepcopy(t1.adv_target_hit)

        # Miss attack Untargeted
        t1.atk_params.set_params(batch_size=batch_size, iteration = 10,
                       target = -1, x_val_min = torch.min(data_x), x_val_max = torch.max(data_x),
                       step_size = 0.01, step_norm = "inf", eps = eps, eps_norm = 2)
        t1.generate_xadv(atk_type = "pgd")
        t1.send_to_victims(victim_idxs)
        logs_adv[adv_idx]['adv_miss'] = copy.deepcopy(t1.adv_acc_transfers)
        logs_adv[adv_idx]['adv_similarities_untarget'] = copy.deepcopy(t1.adv_similarities)
        
    return logs_adv

def logs_adv_analysis(logs_adv):
    
    logs_analyzed_dict = {}
    
    new_num_models = len(logs_adv)
    num_models = len(logs_adv)
    
    metrics = ['orig_acc_transfers','orig_similarities','adv_acc_transfers','adv_similarities_target',
           'adv_similarities_untarget','adv_target','adv_miss'] #,'metric_alignment']

    orig_acc = np.zeros([new_num_models, new_num_models]) 
    orig_sim = np.zeros([new_num_models, new_num_models]) 
    adv_acc = np.zeros([new_num_models, new_num_models]) 
    adv_sim_target = np.zeros([new_num_models, new_num_models]) 
    adv_sim_untarget = np.zeros([new_num_models, new_num_models]) 
    adv_target = np.zeros([new_num_models, new_num_models])
    adv_miss = np.zeros([new_num_models, new_num_models]) 

    for adv_idx in range(num_models):
        for victim in range(num_models):
            orig_acc[adv_idx,victim] = logs_adv[adv_idx][metrics[0]][victim].data.tolist()
            orig_sim[adv_idx,victim] = logs_adv[adv_idx][metrics[1]][victim].data.tolist()
            adv_acc[adv_idx,victim] = logs_adv[adv_idx][metrics[2]][victim].data.tolist()
            adv_sim_target[adv_idx,victim] = logs_adv[adv_idx][metrics[3]][victim].data.tolist()
            adv_sim_untarget[adv_idx,victim] = logs_adv[adv_idx][metrics[4]][victim].data.tolist()
            adv_target[adv_idx,victim] = logs_adv[adv_idx][metrics[5]][victim].data.tolist()
            adv_miss[adv_idx,victim] = logs_adv[adv_idx][metrics[6]][victim].data.tolist()
            
        
    logs_analyzed_dict['orig_acc'] = orig_acc
    logs_analyzed_dict['orig_sim'] = orig_sim
    logs_analyzed_dict['adv_acc'] = adv_acc
    logs_analyzed_dict['adv_sim_target'] = adv_sim_target
    logs_analyzed_dict['adv_sim_target'] = adv_sim_untarget
    logs_analyzed_dict['adv_target'] = adv_target
    logs_analyzed_dict['adv_miss'] = adv_miss
    
    return logs_analyzed_dict

def measure_IB(models_test, dataloader, num_trials = 20, batch_size = 300):
    
    print("IB TEST")
    dists_measure_legit = np.zeros([len(models_test),num_trials, len(models_test)])
    dists_measure_adv = np.zeros([len(models_test),num_trials, len(models_test)])

    # attack parameters -- hard wired for now
    eps = 4.5
    iteration = 10
    target = -1
    eps_norm = 2
    step_size = 0.01

    t1 = Boundary_Transferer(models_list=models_test, dataloader=dataloader)

    for j in range(len(models_test)):

        for i in range(num_trials):
            print("basemodel:", j, "IB num_trial:", i)
            t1.base_nn_idx = j
            t1.victim_idx = range(len(models_test))

            t1.atk_params = IFSGM_Params()
            t1.atk_params.set_params(batch_size=1, eps=0.1, alpha=0.01, iteration = 30,
                               target = -1, x_val_min = torch.min(data_x), x_val_max = torch.max(data_x))

            t1.set_adv_NN(t1.base_nn_idx)

            base_ep_legit, victim_eps_legit = t1.legitimate_direction(batch_size=batch_size, ep_granularity = 0.1, 
                                                                      rep_padding = 1000, new_point = True,print_res = False)

            base_ep_adv, victim_eps_adv = t1.adversarial_direction(ep_granularity = 0.1, 
                                                                      rep_padding = 1000, new_point = False,print_res = False)

            idx = 0
            for key, value in victim_eps_legit.items():
                dists_measure_legit[j,i,idx] = np.abs(base_ep_legit-value)
                idx+=1

            idx = 0
            for key, value in victim_eps_adv.items():
                dists_measure_adv[j,i,idx] = np.abs(base_ep_adv - value)
                idx+=1
    
    # Post filtering
    # Filter data for adv and legit

    for i in range(len(models_test)):
        for j in range(dists_measure_legit[i].shape[0]):
            if dists_measure_legit[i][j][i] > 0:
                dists_measure_legit[i][j][i] = 0
        for j in range(dists_measure_legit[i].shape[0]):
            if dists_measure_adv[i][j][i] > 0:
                dists_measure_adv[i][j][i] = 0

#     ib_legit = np.mean(filtered_dists_measure_legit,axis=0)
#     ib_adv = np.mean(filtered_dists_measure_adv,axis=0)
    return dists_measure_legit, dists_measure_adv

In [5]:
base_name = 'weights/cifar10/client_count/'
directory_names = ['fedEM_c2/','fedEM_c4/','fedEM_c8/',
                   #'fedEM_c10/', 'fedEM_c20/', 'fedEM_c40/',
                   'fedEM_adv_c2/', 'fedEM_adv_c4/', 'fedEM_adv_c8/',
                   #'fedEM_adv_c10/', 'fedEM_adv_c20/', 'fedEM_adv_c40/',
                   'fedEM_dverge_c2/', 'fedEM_dverge_c4/', 'fedEM_dverge_c8/']
                   #,'fedEM_dverge_c10/', 'fedEM_dverge_c20/', 'fedEM_dverge_c40/']

num_models_list = [2,4,8,2,4,8,2,4,8]

# IB test parameters
ib_model_threshold = 10 # Number of models to use for IB analysis
num_trials = 10
batch_size = 200

# Store values
var_dict = {}
orig_acc_dict = {}
adv_miss_dict = {}
ib_dist_dict = {}
ib_dist_dict_compressed = {}

In [ ]:
logs_dict = {}
for i in range(len(directory_names)):
     
    print("EXP:", directory_names[i][:-1])
    weights, models_test, h_s, clients = get_models_test(base_name+directory_names[i], num_models_list[i])
    logs_adv = adv_atk(models_test, num_models_list[i], clients, custom_batch_size = 500)
    logs_dict[directory_names[i][:-1]] = logs_adv_analysis(logs_adv)
    
#     Interboundary dist
    ib_leg, ib_adv = measure_IB(h_s, dataloader, num_trials=4)
    
    # Extract Values of Relevance
    orig_acc = logs_dict[directory_names[i][:-1]]['orig_acc']
    orig_acc_dict[directory_names[i][:-1]] = np.mean(np.diagonal(orig_acc))
    
    adv_miss = logs_dict[directory_names[i][:-1]]['adv_miss']
    adv_miss_dict[directory_names[i][:-1]] = np.mean(np.diagonal(adv_miss))
    
    var = np.var(weights, axis=0)
    var_dict[directory_names[i][:-1]] = var

    ib_dist_dict[directory_names[i][:-1]] = (np.mean(ib_leg, axis=1), np.mean(ib_adv,axis=1))
    ib_dist_dict_compressed[directory_names[i][:-1]] = (np.mean(ib_leg), np.mean(ib_adv))

In [ ]:
var_dict

In [ ]:
for key in var_dict:
    var_mean = np.mean(var_dict[key])
    print(key, var_mean)


In [ ]:
orig_acc_dict

In [ ]:
adv_miss_dict

In [ ]:
ib_dist_dict_compressed